# 0. 라이브러리 불러오기

In [1]:
%%capture
! pip install -U prophet

In [2]:
import datetime
import random
from tqdm.notebook import tqdm
from glob import glob
import gc; gc.enable()

import pandas as pd
import warnings; warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from prophet import Prophet
from sklearn.preprocessing import StandardScaler

from google.colab import drive
ROOT = '/content/drive/'; drive.mount(ROOT)
PATH = f'{ROOT}/MyDrive/Dacon/GrowRegression/'

%cd $PATH
%ls

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1xwyT2VNzWX8ilYppxpO9oeeGzeFniDyZ/GrowRegression
data/  Notebook/  output/  reference/  submission/


In [3]:
# 시드 고정
random.seed(42) 
np.random.seed(42)

# 1. 이상치 탐색 Anomaly Detection

- 알고리즘 : Meta(Facebook)'s Prophet
    - Parameters : 계절성(seasonality) ➡ daily만 고려
        - 하루를 cycle로 구성함
    - Computation   
⭐ uncertainty ➡ 예측치 $\widehat{y}$의 upper bound - lower bound   
⭐ anomaly ➡ $|y-\widehat{y}| \ge α*uncertainty$   
⭐ $α$는 3.5 정도의 값으로 설정   
⭐ anomaly로 설정된 값은 결측치 처리 ➡ 이후 imputation을 수행 할 예정.


***Note.***   
- Mean $\pm$ SD (M/SD): 값이 0인 경우가 많아 (ex. 야간의 식물재배등 광량) mean이 지나치게 낮게 설정되어 outlier가 과도하게 설정됨.
- Hampel Filter : 특정 window size에 기반하여 meidan $\pm$ SD로 추정하는 방식. M/SD의 경우보다는 적은 수의 표본이 anomaly로 설정되나 이상치가 연속적으로 나오는 경우(고의적으로 넣은 것으로 추정되는 수치)에 반응하지 못함. window size를 크게 잡은 경우(4시간, 6시간, 24시간 등)에는 M/SD와 거의 동일한 결과.
    

In [ ]:
# 파일 경로 가져오기
trainList = sorted(glob("data/data_rounded/train*"))
testList = sorted(glob("data/data_rounded/test*"))

In [ ]:
# set Prophet verbosity as 'False'
import logging; logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

def ProphetAnomaly(dataset:pd.DataFrame, feature:str, anomalyCoef:float=3.5) -> pd.DataFrame:
    # Fitting Prophet
    y = dataset[feature].reset_index()
    y.columns = ['ds', 'y']
    m = Prophet(
        changepoint_range=0.95,
        yearly_seasonality=False,
        weekly_seasonality=False,
                )
    m.fit(y)
    future = m.make_future_dataframe(periods=360, freq='min')
    forecast = m.predict(future)

    # Find Anomaly
    result = pd.concat([y.set_index('ds')['y'], forecast.set_index('ds')[['yhat','yhat_lower','yhat_upper']]], axis=1)
    result['error'] = result['y'] - result['yhat']
    result['uncertainty'] = result['yhat_upper'] - result['yhat_lower']
    result['anomaly'] = result.apply(lambda x: 1 if(np.abs(x['error']) >= anomalyCoef * x['uncertainty']) else 0, axis = 1)
    result['anomaly'] = result['anomaly'].astype('int')
    
    result = result.loc[y.ds.values, :]

    return result

## Sample Output

In [ ]:
sampleDataset = pd.read_parquet(trainList[31])
res = ProphetAnomaly(dataset=sampleDataset, 
                     feature='WhiteLEDRemains',
                     anomalyCoef = 3.5)

res['anomaly'] = res['anomaly'].astype('str') # for visualization

fig = px.scatter(
    res.reset_index(), x='ds', y='y', color='anomaly'
)

fig.update_layout(
    title = {
        'text' : "Anomaly Detection Results of White LED Remains",
        'x' : 0.5
        },
    
    xaxis_title = {
        'text' : 'Time'
    },
    
    yaxis_title = {
        'text' : 'White LED Remains'
    },
                  )

fig.show()

DEBUG:cmdstanpy:input tempfile: /tmp/tmpb0kdlzxb/8tfbjfwv.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpb0kdlzxb/7ozuk4y9.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=92467', 'data', 'file=/tmp/tmpb0kdlzxb/8tfbjfwv.json', 'init=/tmp/tmpb0kdlzxb/7ozuk4y9.json', 'output', 'file=/tmp/tmpaj6cv84p/prophet_model-20220907170928.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:09:28 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:09:34 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


## 결측치 채우기

- 여러가지 케이스로 구성된 시계열 데이터이므로, 날짜를 헤아리지 않고 시점 $t_{i}$의 모든 데이터 평균을 구하여 대체.
- train 데이터에서 구해진 값으로 (만일 존재한다면) test 데이터에 존재하는 결측치를 처리함. (data-leakage 방지)
- 각 데이터셋의 길이가 다르므로 모두 더하여 일괄적으로 나누는 대신 등장 회수를 고려하여 나누어줌

In [ ]:
firstFile = True
for filePath in tqdm(trainList):
    F = pd.read_parquet(filePath).reset_index(drop=True)
    F = F.astype('float64')

    if firstFile:
        trainMean = F
        trainCounter = F.copy()
        trainCounter[trainCounter!=-123415111] = 1 # 데이터를 횟수 1로 바꾸어줌
        firstFile = False
    else:
        trainMean = trainMean.add(F, fill_value=0)
        F_Count = F.copy()
        F_Count[F_Count!=-123415111] = 1
        trainCounter = trainCounter.add(F_Count, fill_value=0)

trainMean = trainMean.divide(trainCounter, fill_value=0)

  0%|          | 0/58 [00:00<?, ?it/s]

##결측치 채우기 & 이상치 탐지

실행이 오래 걸리므로 나누어 수행함.

In [ ]:
for filePath in tqdm(trainList+testList):
    F = pd.read_parquet(filePath).reset_index()
    features = [x for x in F.columns if x not in ['time']]
    for feature in tqdm(features, leave=False):
        # impute missing value by global mean
        missingIdx = F[F[feature].isna()].index.tolist()
        F.loc[missingIdx, feature] = trainMean.loc[missingIdx, feature]
        # detect anomaly
        F = F.set_index('time')
        res = ProphetAnomaly(dataset=F, 
                             feature=feature,
                             anomalyCoef = 3.5)
        res['anomaly'] = res['anomaly'].apply(lambda x: np.nan if (x==1) else 1)
        res['y'] = res['y'] * res['anomaly']
        F[feature] = res['y'] # replace anomaly as np.nan
        # fill value by interpolation
        if F[feature].isna().sum() > 0:
            F[feature] = F[feature].interpolate(method='linear', axis=0) 
        else:
            continue
        F = F.reset_index()
        
    
    F.to_parquet(f"data/data_imputed/{filePath.split('/')[-1]}")

# 2. Target Value 이상치 확인

## 살펴보기

In [ ]:
labelList = sorted(glob("data/data_raw/train_target/*"))
iterLab = iter(labelList)

In [ ]:
filePath = next(iterLab)
lab = pd.read_csv(filePath, parse_dates=['시간'])
fig = px.line(data_frame=lab, x=lab.index, y='rate')

fig.update_layout(
    title = {
        'text' : f"Target Value of {filePath.split('/')[-1].split('.')[0]}",
        'x' : 0.5
        },
    
    xaxis_title = {
        'text' : 'Time'
    },
    
    yaxis_title = {
        'text' : 'Growth Rate'
    },
                  )

fig.show()

- Case 27에서 확인되듯 성장률 데이터에도 이상치가 확인됨
- 앞선 경우와 마찬가지로 이상치에 대해서는 결측치로 처리한 뒤 보간법(interpolation)으로 채워줌.
- 현재 타겟 변수인 rate가 전일 대비 잎 면적의 증감률을 나타냄
- 정확히 어느 생육단계인지 (발아 초기-유묘-중묘 등) 확인이 안되므로 초기 단계에 가능 할 것으로 고려되는 1000% 증가(데이터에서는 10)를 기준으로 그 이상은 이상치로 처리함. 
    - 싹이 터서 막 올라온 상태였다고 가정 ➡ 이파리가 세로로 서있어 아주 작게 측정되는 경우도 고려.
    - Prophet 등 시계열 모형을 사용하기에는 각 라벨 데이터의 개수는 30개 내외로 부족함.
- 이상치는 missing value로 처리하고 보간법(interpolation)으로 채움.
    - 가장 첫번째/마지막 측정치는 보간법이 불가능함으로 제외 

## 보간법 (Interpolation)

In [ ]:
target = []
for filePath in labelList:
    lab = pd.read_csv(filePath, parse_dates=['시간'])
    lab.columns = ['time', 'rate']
    lab = lab.set_index('time')
    lab['rate'].iloc[1:-1] = lab['rate'].iloc[1:-1].apply(lambda x: np.nan if (x > 10) else x)
    lab['rate'] = lab['rate'].interpolate(method='linear', axis=0)
    lab.to_parquet(f"data/data_label/{filePath.split('/')[-1].split('.')[0]}.parquet")

# 3. Data Aggregation

- 데이터의 수가 많은 것은 좋으나, 컴퓨팅 파워를 고려하여 데이터를 aggrgation 하도록 함.

- 1시간 단위로 데이터를 aggregation하되, 이동평균의 경우 결측치가 발생하므로 단순평균을 연산함.

- 10분 등의 단위의 경우 타겟 변수를 증강(augmentation) 했을 경우 너무 작은 값들이 분포하게됨.

In [4]:
trainList = sorted(glob("data/data_imputed/train*"))
testList = sorted(glob("data/data_imputed/test*"))

for filePath in trainList + testList:
    F = pd.read_parquet(filePath)
    F = F.set_index('time')
    F = F.resample('1h').mean()
    F.to_parquet(f"data/data_agg/{filePath.split('/')[-1]}")

# 4. 타겟 변수 증강 (Target Value Augmentation)

## Main Idea
- 입력 데이터는 분 단위이나 타겟 변수인 성장률은 일 단위로 기록되어 있으므로 augmentation을 통해서 분단위에 적합하게 증강(augmentation)
- 이론적으로 식물은 빛이 거의 없는 야간에 성장함 ➡ 총광량인 Total Light의 값에 따라서 타겟 변수를 증강
- 식물의 생리상 빛이 없는 구간에서 항상 동일하게 상수만큼 성장하는 것이 아닌 선형적인 증가-감소가 발생함.
- 이를 반영하기 위해서는 구간별로 다른 값을 지녀야하며, 인덱스가 단조증가가 아닌 증감을 모두 반영 할 수 있는 형태여야함.
- 다음의 연산으로 위의 사항들을 반영하고자함.
## Formula
    1. 일단위로 데이터를 분리
    2. 정수 인덱스를 추출
    3. 추출된 정수 인덱스에 대하여 다음의 연산을 통하여 변환   
--- 
- t는 일(day), i는 시(hour)을 의미
- $A_{t}$는 인덱스 i로 구성된 t 시점의 Array를 의미
- $L_{i}$는 인덱스 i의 시점에 대해 광량을 변환한 값
    - 광량(Total Light)를 X라고 할 때,
    - X가 0일 경우 1
    - X이 0보다 클 경우 $\frac{1}{X_{i}}$

- $R_{t}$은 타겟 변수의 하루치 값을 의미.   
$$\frac{|(A_{i}*L_{i})\ -\ \lfloor(length(A) / 2)⌋|}{ΣA_{i}} \ \ \  \ \ \ \ \times \ R_{t}$$
--- 

- 연산의 결과는 다음으로 해석 할 수 있음
    - 식물의 성장이 빛이 없을 때에만 일어난다고 가정 할 때, 
    - 시점 t의 총성장률 $R_{t}$를 빛이 없는 구간 $L_{i}$에 대하여 데이터를 새롭게 생성함 (증강함)

## 함수 정의

In [6]:
# 파일 경로 가져오기
trainList = sorted(glob("data/data_agg/train*"))
testList = sorted(glob("data/data_agg/test*"))
labelList = sorted(glob("data/data_label/*"))

In [7]:
def LabelAugmentation(inputPath:str, labelPath:str): 
    inputData = pd.read_parquet(inputPath)
    label = pd.read_parquet(labelPath)
    timeDaily = label.index - pd.Timedelta(days=1)
    timeDaily = timeDaily.tolist()
    timeDaily.append(label.index.tolist()[-1])
    
    y_aug = []
    for i in range(1, len(timeDaily)): 
        tmp = inputData.loc[timeDaily[i-1]:timeDaily[i]-pd.Timedelta(minutes=1)]
        tmp_rate = label['rate'][i-1]
        lightOff = [1 if(x==0) else 1/x for x in tmp['TotalLight']]
        idx = [abs(_-(len(tmp)//2)) for _ in [x for x in range(len(tmp))]]
        y = [lightIdx * j for j, lightIdx in zip(idx, lightOff)]
        y = np.array([k/sum(y) for k in y]) * tmp_rate
        y_aug.extend(y)
    
    inputIdx = inputData.index
    return inputIdx, y_aug

## 결과 예시 (Case 32)

In [8]:
sampleIdx, sampleLabelAug = LabelAugmentation(trainList[32], labelList[32])
sampleAug = pd.DataFrame([sampleIdx, sampleLabelAug]).T
sampleAug.columns=['time', 'rate']
sampleAug = sampleAug.set_index('time')

fig = px.line(data_frame=sampleAug, x=sampleAug.index, y='rate')
fig.show()

## Aggregraion 데이터에 추가

- train 43번 데이터의 첫날을 삭제 할 필요가 있음 ➡ 라벨보다 하루가 많음

In [9]:
t43 = trainList[42]
t = pd.read_parquet(t43)
t = t.loc[datetime.datetime(2022, 3, 22):]
t.to_parquet(t43)

del t, t43

In [10]:
for trainPath, labelPath in zip(trainList, labelList):
    idx, lab = LabelAugmentation(trainPath, labelPath)
    labAug = pd.DataFrame([idx, lab]).T
    labAug.columns=['time', 'rate']
    labAug = labAug.set_index('time')

    inputData = pd.read_parquet(trainPath)
    inputData = pd.merge(left=inputData, right=labAug, 
                         left_index=True, right_index=True,
                         how = 'left')
    
    inputData.to_parquet(trainPath)

# 5. 시차(Lag)

- 이후 사용될 모델이 전통적 시계열 모형(Prophet, ARIMA 등)이 아닌 머신러닝/신경망을 사용 할 것이므로 Stationarity 가정을 완벽하게 지키기보다는 시차 변수만을 추가함.

- 변환
    - 시차 범위는 변수 시점 기준 전/후 24시간   
    - 발생하는 missing value는 0으로 채움


In [11]:
# 파일 경로 업데이트
trainList = sorted(glob("data/data_agg/train*"))
testList = sorted(glob("data/data_agg/test*"))

In [12]:
def AddLag(filePath):
    F = pd.read_parquet(filePath)
    df = F.copy()
    
    for feature in [x for x in df.columns if x not in ['rate']]:
        
        for lag in [x for x in range(-24, 25) if x != 0]: 
            df[f"{feature}_lag_{lag}"] = df[feature].shift(lag)        

    return df

In [13]:
for filePath in trainList + testList:
    F = AddLag(filePath)
    F.to_parquet(filePath)

# 6. 새 변수 추가 : Day


- LightGBM 등의 모델은 경과를 반영하지못하므로 데이터가 수집된 시점으로부터 경과 추가 


In [14]:
for filePath in tqdm(trainList + testList):
    F = pd.read_parquet(filePath)
    if 'day' in F.columns:
        F = F.drop('day', axis=1)

    date = F.index
    dateDiff = date - date[0]
    dateDiff = dateDiff.days + 1
    F.insert(0, 'day', dateDiff)

    F.to_parquet(filePath)

  0%|          | 0/64 [00:00<?, ?it/s]

# 7. K-Fold Dataset

시계열-패널 데이터로 생각하여 케이스 단위로 K-fold를 수행하고, 모델 학습시간 단축을 위해서 데이터를 미리 생성함.

In [15]:
trainList = sorted(glob("data/data_agg/train*"))
labelList = sorted(glob("data/data_label/*"))
testList = sorted(glob("data/data_agg/test*"))

In [16]:
# 시계열 K-fold
def KsplitTrainValid(pathList:list, validSize:int=11):
    idxList = []
    idx = [i for i in range(len(pathList))]

    start = 0
    for i in range(1, len(pathList)//10+1):
        validIdx = list(range(start, i*validSize))
        trainIdx = [x for x in range(len(pathList)) if x not in validIdx]
        idxList.append((trainIdx, validIdx))
        start += validSize
        
    return list(reversed(idxList))

In [17]:
KFoldIdx = KsplitTrainValid(trainList, validSize=11)

In [18]:
for k, Idx in enumerate(KFoldIdx):
    fold = k+1
    trIdx, valIdx = Idx

    print('*'*50)
    print(f"Create Fold {fold} dataset")
    
    train, valid = pd.DataFrame(), pd.DataFrame()
    trainLabel, validLabel = pd.DataFrame(), pd.DataFrame()

    for idx in tqdm(trIdx, leave=False):
        F = pd.read_parquet(trainList[idx])
        F['ID'] = idx+1
        train = pd.concat([train, F])
        
        F_label = pd.read_parquet(labelList[idx])
        F_label['ID'] = idx+1
        trainLabel = pd.concat([trainLabel, F_label])
        
        del F, F_label; gc.collect()

    train.to_parquet(f'data/data_kfold/input/train_{fold}.parquet')
    trainLabel.to_parquet(f'data/data_kfold/label/train_{fold}.parquet')

    for idx in tqdm(valIdx, leave=False):
        F = pd.read_parquet(trainList[idx])
        F['ID'] = idx+1 
        valid = pd.concat([valid, F])

        F_label = pd.read_parquet(labelList[idx])
        F_label['ID'] = idx+1
        validLabel = pd.concat([validLabel, F_label])
        
        del F, F_label; gc.collect()
    
    valid.to_parquet(f'data/data_kfold/input/valid_{fold}.parquet')
    validLabel.to_parquet(f'data/data_kfold/label/valid_{fold}.parquet')

**************************************************
Create Fold 1 dataset


  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

**************************************************
Create Fold 2 dataset


  0%|          | 0/47 [00:00<?, ?it/s]

...

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

**************************************************
Create Fold 5 dataset


  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]